## Part 1
Time evolution of position and velocity:
$$
\vec{x}_t = \vec{x}_{t-1} + \vec{v}_{t-1}\cdot\Delta t \\ 
\vec{v}_t = \vec{v}_{t-1} + \vec{g}\cdot\Delta t + \vec{d}{} (\vec{v}_{t-1})
$$

where $\vec{g}=\begin{pmatrix} 0 & 1 \end{pmatrix}$ and $\vec{d(\vec{v}_{t-1})}=\begin{pmatrix} \mathrm{sgn}(v_{x,t-1}) & 0 \end{pmatrix}$ and a time step of $\Delta t=1$.

Iterative insertion yields the solutions:
$$
\vec{x}_t = \sum_{\tau=0}^{t-1} \vec{v}_\tau \\ 
\vec{v}_t = \vec{v}_0 + t\cdot\vec{g} + \sum_{\tau=0}^{t-1}\vec{d}(\vec{v}_\tau)
$$

For the $y$-component of $\vec{x}$, the solution can be directly expressed as:
$$
y_{t} = \sum_{\tau=0}^{t-1}(v_{y,0} - \tau) = t\cdot v_{y,0} - \sum_{\tau=0}^{t-1}\tau = t\cdot v_{y,0} - \frac{t(t-1)}{2}
$$
which is a discrete parabola with two maxima for $t=v_{y_0}$ and $t=v_{y_0}+1$, the maximum height is therefore given as $y_\mathrm{max}=\frac{1}{2}v_{y_0}(v_{y_0}+1)$.

After a time $t_0 = 2v_{y,0}+1$ the original height $y=0$ is reached again, and the motion continues downwards with $v_{y,t_0}=v_{y_0}-(2v_{y_0} +1)=-v_{y_0}-1$.
The highest velocity allowed at this point is actually just given as $y_\mathrm{min}$, assuming the target area fulfills $y_\mathrm{min} < 0$, as otherwise we cannot hit the target area:

$v_{y,t_0}=y_\mathrm{min}=-v_{y_0}-1=>v_{y_0}=-y_\mathrm{min}-1$.

The correspondig height is thous given as $y_\mathrm{max}=\frac{1}{2}(-y_\mathrm{min}-1)(-y_\mathrm{min}-1+1) = \frac{1}{2}y_\mathrm{min}(y_\mathrm{min}+1)$.

In [1]:
# target area: x=20..30, y=-10..-5
# target area: x=282..314, y=-80..-45
ymin = -80
ymax = -45
print("Part 1:", 0.5*ymin*(ymin+1))

Part 1: 3160.0


## Part 2

In [2]:
import numpy as np

xmin=282
xmax=314
ymin=-80
ymax=-45

def d(v):
    return np.array([-np.sign(v[0]), 0])

g = np.array([0,-1])

class Vt:
    """
    Avoid recalculating timesteps that have already been solved by storing the state of v and t
    """
    def __init__(self, v0):
        self.v0 = v0
        self.v = [v0]
        self.t = 0
    
    def __call__(self, t):
        if t < self.t:
            return self.v[:t+1]
        for tau in range(self.t, t):
            self.v.append(self.v[-1]+g+d(self.v[-1]))        
        self.t = t
        return self.v

def xt(t, vt):
    return np.sum(vt(t), axis=0)

def hit(x, xmin, xmax, ymin, ymax):
    return (xmin <= x[0] <= xmax) and (ymin <= x[1] <= ymax)

def fail(x, xmax, ymin):
    return  (x[0] > xmax) or (x[1] < ymin)


Some bounds on the velocity: The highest reached x position will be

$$x = \sum_{\tau=0}^{v_{x,0}-1} (v_{x,0}-\tau)= v_{x,0}^2 - \frac{v_{x,0}(v_{x,0}-1)}{2}=\frac{v_{x,0}(v_{x,0}+1)}{2} = \binom{v_{x,0}}{2} $$
which needs to be at least $x_\mathrm{min}$. Solving the resulting quadratic equation for $v_{x,0}$ yields
$$v_{x,0} \geq \frac{1}{2}(\sqrt{8x_\mathrm{min}+1}-1)$$
At the same time, $v_{x,0}$ cannot be more than $x_\mathrm{max}$, as we would overshoot the target area otherwise.

For $v_{y,0}$, a similar lower boundary exists as $y_\mathrm{min}$, and the maximum velocity from part 1: $v_{y,0,\mathrm{max}}=-y_\mathrm{min}-1$

In [3]:
v0xmin = int(np.ceil(0.5*(np.sqrt(8*xmin+1)-1)))
v0xmax = xmax
v0ymin = ymin
v0ymax = -ymin-1
grid = np.meshgrid(list(range(v0xmin, v0xmax+1)), list(range(v0ymin, v0ymax+1)))
velocities = np.array(list(zip(*(g.flat for g in grid))))
good_v = []
vt = Vt((5, 3160))
for v0 in velocities:
    vt = Vt(v0)
    t = 0
    while True:
        x = xt(t, vt)
        if hit(x, xmin, xmax, ymin, ymax):
            
            good_v.append(v0)
            break
        if fail(x, xmax, ymin):
            break
        t += 1

print("Part 2:", len(good_v))

Part 2: 1928
